## Calculo de tiempo promedio de horas pico de las sucursales 
### Desarrollado por:
- **Nombre:** Gabriel Chavez 
- **Fecha:** 2024-11-21

### Descripción
Tomaremos de referencia las transacciones de kielsa Honduras, donde pondremos de forma ordenada las cajas y los id de facturas, para poder calcular el tiempo promedio de horas pico de las sucursales.
- La hora de inicio sera la fecha de finazalizacion anterior.
- El tiempo promedio sera la diferencia entre la hora de inicio y la hora de fin de la transaccion.
- Organizar por tipo de transaccion y sacar el promedio.


In [2]:
import polars as pl
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import text

from sqlalchemy import create_engine
from urllib.parse import quote_plus


connection_string = (
    'mssql+pyodbc://'
    'Angel_chavez:{}@172.16.2.227\\DWHFARINTERDEV/BI_FARINTER?'
    'driver=ODBC+Driver+17+for+SQL+Server'.format(quote_plus('@ng3l_ch@v3z'))
)
engine = create_engine(connection_string)
query = text("""SELECT
    fc.Suc_Id,
    fc.Emp_Id,
    fc.Caja_Id,
    fc.Factura_Id,
    fc.Factura_Fecha,
    fc.Factura_FechaHora,
    fc.AnioMes_Id,
    fc.Cliente_Id,
    fc.MonederoTarj_Id,
    fc.Vendedor_Id,
    CASE 
        WHEN fc.Monedero_Id = '0' THEN 'No Identificada'
        ELSE md.Tipo_Plan 
    END AS Tipo_Plan,
    md.Edad,
    suc.TipoSucursal_Id,
    suc.TipoSucursal_Nombre,
    suc.Zona_Id,
    suc.Zona_Nombre
FROM
    [BI_Kielsa_Hecho_FacturaEncabezado] AS fc
    JOIN [BI_Kielsa_Dim_Monedero] AS md 
        ON md.Monedero_Id = fc.Monedero_Id AND md.Emp_Id = fc.Emp_Id       
    JOIN [BI_Kielsa_Dim_Sucursal] AS suc 
        ON suc.Sucursal_Id = fc.Suc_Id AND suc.Emp_Id = fc.Emp_Id
WHERE
    fc.AnioMes_Id = 202410 
    AND fc.Emp_Id = 1;""")

with engine.connect() as conn:
    df_base = pd.read_sql(query, conn).set_index('Factura_Id').reset_index()



In [3]:
df_base.head()

,Factura_Id,Suc_Id,Emp_Id,Caja_Id,Factura_Fecha,Factura_FechaHora,AnioMes_Id,Cliente_Id,MonederoTarj_Id,Vendedor_Id,Tipo_Plan,Edad,TipoSucursal_Id,TipoSucursal_Nombre,Zona_Id,Zona_Nombre
0,98269,248,1,1,2024-10-26,12:02:31,202410,10000001,0502-2000-01595,10004684,Monedero todo publico,24,9,Plaza Comercial,2,NORTE
1,174059,137,1,3,2024-10-05,14:57:31,202410,10000001,0502-2000-01612,10000001,Monedero todo publico,24,7,Stand Alone,2,NORTE
2,174051,137,1,3,2024-10-04,19:35:01,202410,10000001,0502-2000-01637,10002549,Monedero todo publico,24,7,Stand Alone,2,NORTE
3,58162,36,1,3,2024-10-26,12:44:12,202410,10000001,0502-2000-01771,10003924,Monedero todo publico,24,2,Centro Comercial,2,NORTE
4,57556,36,1,3,2024-10-18,16:11:50,202410,10000001,0502-2000-01797,10003924,Monedero todo publico,25,2,Centro Comercial,2,NORTE


In [4]:
# Asegurarse de que Factura_Fecha sea tipo datetime



In [5]:
import numpy as np

# Copia de DataFrame base
df = df_base.copy()

tipos_transaccion = {
    'Monedero tercera edad': 'Tercera Edad',
    'Monedero Cuarta Edad': 'Tercera Edad',
    'Monedero todo publico': 'Todo Publico',
    'Monedero Clínica TP': 'Todo Publico',
    'Monedero Clínica TE': 'Tercera Edad',
    'No Identificada': 'No Identificada',
}

# Mapear los tipos de transacciones y eliminar las no relevantes
df['Tipo_Plan'] = df['Tipo_Plan'].map(tipos_transaccion)
df = df.dropna(subset=['Tipo_Plan'])


# Convertir Factura_FechaHora a cadena para agregar fecha ficticia
df['Factura_FechaHora'] = df['Factura_FechaHora'].astype(str)
df['Factura_FechaHora'] = '1900-01-01 ' + df['Factura_FechaHora']

# Convertir Factura_FechaHora a tipo datetime
df['Factura_FechaHora'] = pd.to_datetime(df['Factura_FechaHora'], errors='coerce')

# Asegurarse de que Factura_Fecha sea tipo datetime
df['Factura_Fecha'] = pd.to_datetime(df['Factura_Fecha'])

# Ordenar el DataFrame
df = df.sort_values(by=['Suc_Id', 'Caja_Id', 'Factura_Fecha', 'Factura_FechaHora'], ascending=True)

# Crear columna Factura_Inicio_FechaHora solo si la fecha, la sucursal y la caja son las mismas
df['Factura_Inicio_FechaHora'] = np.where(
    (df['Suc_Id'] == df['Suc_Id'].shift(1)) & 
    (df['Caja_Id'] == df['Caja_Id'].shift(1)) & 
    (df['Factura_Fecha'] == df['Factura_Fecha'].shift(1)),
    df['Factura_FechaHora'].shift(1),
    pd.NaT  # Asignar NaT si no coinciden
)

# Asegurarse de que la columna Factura_Inicio_FechaHora sea tipo datetime
df['Factura_Inicio_FechaHora'] = pd.to_datetime(df['Factura_Inicio_FechaHora'], errors='coerce')

# Calcular el tiempo transcurrido
df['Tiempo_Transcurrido'] = df['Factura_FechaHora'] - df['Factura_Inicio_FechaHora']

# convertir el tiempo transcurrido a minutos
df['Tiempo_Transcurrido'] = df['Tiempo_Transcurrido'].dt.total_seconds() / 60
# excluir columnas con un Factura_Inicio_FechaHora igual a NaT
df = df.dropna(subset=['Factura_Inicio_FechaHora'])



In [28]:
# Agregar una columna para la hora
df['Hora'] = df['Factura_FechaHora'].dt.hour
# Agrupar por hora y tipo (por ejemplo, Tipo_Plan) y calcular métricas
resultados = df.groupby(['Hora', 'Tipo_Plan', 'Suc_Id', 'Caja_Id', 'Factura_Fecha']).agg(
    Tiempo_Promedio=('Tiempo_Transcurrido', 'mean'),
    Cantidad_Transacciones=('Factura_Id', 'count')
).reset_index()

resultados.Tiempo_Promedio.max()

1270.6666666666667

In [29]:
# Identificar horas pico basadas en el mayor número de transacciones
horas_pico_por_sucursal_dia = resultados.groupby(['Suc_Id', 'Factura_Fecha', 'Hora']).agg(
    Cantidad_Transacciones=('Cantidad_Transacciones', 'sum')
).reset_index()
horas_pico_por_sucursal_dia['Hora_Pico'] = horas_pico_por_sucursal_dia.groupby(['Suc_Id', 'Factura_Fecha'])['Cantidad_Transacciones'].transform(max) == horas_pico_por_sucursal_dia['Cantidad_Transacciones']



In [45]:
print(horas_pico_por_sucursal_dia[horas_pico_por_sucursal_dia['Hora_Pico']])

        Suc_Id Factura_Fecha  Hora  Cantidad_Transacciones  Hora_Pico
6            1    2024-10-01    14                       9       True
19           1    2024-10-02    13                       9       True
24           1    2024-10-02    18                       9       True
38           1    2024-10-03    18                       8       True
49           1    2024-10-04    17                       8       True
...        ...           ...   ...                     ...        ...
119688     328    2024-10-29    19                       3       True
119691     328    2024-10-30     9                       5       True
119695     328    2024-10-30    13                       5       True
119696     328    2024-10-30    14                       5       True
119710     328    2024-10-31    18                       5       True

[11392 rows x 5 columns]


In [31]:
# Filtrar solo las horas pico
horas_pico_solo_dia = horas_pico_por_sucursal_dia[horas_pico_por_sucursal_dia['Hora_Pico']]


In [32]:
# Calcular los promedios por tipo de transacción considerando solo las horas pico
promedios_horas_pico = resultados.merge(
    horas_pico_solo_dia[['Suc_Id', 'Factura_Fecha', 'Hora']],
    on=['Suc_Id', 'Factura_Fecha', 'Hora'],
    how='inner'
)

In [ ]:
# Agrupar por tipo de transacción y calcular el tiempo promedio en horas pico
promedios_horas_pico_general = promedios_horas_pico.groupby(['Suc_Id', 'Tipo_Plan']).agg(
    Tiempo_Promedio=('Tiempo_Promedio', 'mean')
).reset_index()

print("Promedio de tiempo en horas pico por tipo de transacción y sucursal")
print(promedios_horas_pico_general)


In [43]:
promedio_general_por_tipo = promedios_horas_pico_general.groupby('Tipo_Plan').agg(
    Tiempo_Promedio_Tipo=('Tiempo_Promedio', 'mean')
).reset_index()

# Resultados
print("Promedios Generales de Tiempo en Horas Pico por Tipo de Transacción:")
print(promedio_general_por_tipo)

Promedios Generales de Tiempo en Horas Pico por Tipo de Transacción:
         Tipo_Plan  Tiempo_Promedio_Tipo
0  No Identificada             12.992073
1     Tercera Edad             15.183400
2     Todo Publico             14.085435


In [41]:
# Resultados
promedios_horas_pico_por_sucursal = promedios_horas_pico_general.groupby('Suc_Id').agg(
    Tiempo_Promedio_Sucursal=('Tiempo_Promedio', 'mean')
).reset_index()

# Resultados
print("Promedios de Tiempo en Horas Pico por Sucursal:")
print(promedios_horas_pico_por_sucursal)




Promedios de Tiempo en Horas Pico por Sucursal:
     Suc_Id  Tiempo_Promedio_Sucursal
0         1                 20.222171
1         2                 21.626214
2         3                 15.393112
3         4                 12.761191
4         5                 11.645889
..      ...                       ...
293     324                 28.786234
294     325                 28.298066
295     326                 16.885358
296     327                 11.663432
297     328                 31.913566

[298 rows x 2 columns]
